In [1]:
import numpy as np
from math import exp
import matplotlib.pyplot as plt
from matplotlib import animation
from tqdm.notebook import tnrange

In [2]:
K = 1.6e6
E = 8e4
alpha = 1 # непонятно
Q = 7e5
ro = 830
C = 1990
lambd = 0.13
D = 8e-8
R = 8.31
T0 = 293 # начальная температура
Tm = T0 + Q / C # вроде бы конечная температура
kappa = lambd / (ro * C)
U = (2 * K * lambd / (Q * ro * Q / C) * T0 / Tm * ((R * Tm * Tm / E) ** 2) * exp(-E / (R * Tm))) ** 0.5
beta = R * Tm / E
gamma = R * Tm * Tm / (E * Q / C)
delta_h = kappa / U
delta_r = beta * delta_h
delta_d = D / U
Le = delta_d / delta_h

In [3]:
print(Le)

1.0164307692307692


In [4]:
a = 0
b = 3e-2
dz = 1e-3
z_steps = int((b-a) / dz)

teta = 500
dt = 1e-1
t_steps = int(teta / dt)

r_X = D * dt / (dz ** 2)
r_T = kappa * dt / (dz ** 2)

In [5]:
print(r_X, r_T)

0.008 0.007870678694678212


In [6]:
t_start = np.zeros(z_steps, dtype=np.float64)
x_start = np.zeros(z_steps, dtype=np.float64)

for i in range(1):
    t_start[i] = Tm
    x_start[i] = 0
for i in range(1, z_steps):
    t_start[i] = T0
    x_start[i] = 1

In [7]:
def W(X, T):
    return -K*(X ** alpha)*exp(-E/(R*T)) 

In [8]:
def solve(solver):
    Ts = [t_start]
    Xs = [x_start]
    
    for cur_iter in tnrange(1, t_steps + 1):
        t = dt * cur_iter
        cur_t, cur_x = solver(t, Ts, Xs)
        Ts.append(cur_t)
        Xs.append(cur_x)

    return Ts, Xs

In [9]:
def evident_opposite(t, Ts, Xs):
    next_t_row = np.zeros(z_steps, dtype=np.float64)
    next_x_row = np.zeros(z_steps, dtype=np.float64)

    for i in range(1, z_steps - 1):
        prev_t = Ts[-1][i - 1]
        next_t = Ts[-1][i + 1]
        cur_t = Ts[-1][i]
        
        prev_x = Xs[-1][i - 1]
        next_x = Xs[-1][i + 1]
        cur_x = Xs[-1][i]
        
        next_t_row[i] = cur_t + r_T * (prev_t + next_t - 2 * cur_t) - Q/C * dt * W(cur_x, cur_t)
        next_x_row[i] = cur_x + r_X * (prev_x + next_x - 2 * cur_x) + dt * W(cur_x, cur_t)
    
    next_t_row[0] = Tm
    next_t_row[z_steps - 1] = next_t_row[z_steps - 2]
    
    next_x_row[0] = 0
    next_x_row[z_steps - 1] = next_x_row[z_steps - 2]
    return next_t_row, next_x_row

In [ ]:
Ts, Xs = solve(evident_opposite)

In [ ]:
fig, ax = plt.subplots(figsize=(10,5))

x = np.arange(a, b, dz)
line, = ax.plot(x, Ts[0])

def animate(i):
    line.set_ydata(Ts[i])  # update the data
    return line,

# Init only required for blitting to give a clean slate.
def init():
    line.set_ydata(np.ma.array(x, mask=True))
    return line,

ani = animation.FuncAnimation(fig, animate, tnrange(0, t_steps, 50), init, blit=True)
ani.save('./T.gif', writer='pillow', fps=100)

In [ ]:
fig, ax = plt.subplots(figsize=(10,5))

x = np.arange(a, b, dz)
line, = ax.plot(x, Xs[0])

def animate(i):
    line.set_ydata(Xs[i])  # update the data
    return line,

# Init only required for blitting to give a clean slate.
def init():
    line.set_ydata(np.ma.array(x, mask=True))
    return line,

ani = animation.FuncAnimation(fig, animate, tnrange(0, t_steps, 50), init, blit=True)
ani.save('./X.gif', writer='pillow', fps=100)